# Embedding/embedding loss assessment

to do: use loss and test function to fine-tune embeddings, add rare/non-rare consideration

In [1]:
def word_preprocessing(word):
    
    import nltk
    from nltk.stem import WordNetLemmatizer
    # from nltk.corpus import wordnet
    import numpy as np
    import re
    import string
    from nltk.corpus import stopwords

    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('punkt')
    lemma_list = []
    word_list = []

    lemmatizer = WordNetLemmatizer()
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    stop_words = set(stopwords.words('english'))
    
    
    
    lower = word.lower()
    punct_replacer = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    rem_punct = lower.translate(punct_replacer)
    lemma = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(rem_punct)]
    rem_stop = [w for w in lemma if not w in stop_words]
    rem_digits = [re.sub('\d', '<dig>', i) for i in rem_stop]
    lemma_list.append(rem_digits)
    word_list.append(word)
    
    
for i in ["BLue", "Red"]:
    word_preprocessing(i)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ejhaselden/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ejhaselden/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ejhaselden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ejhaselden/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ejhaselden/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ejhaselden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
def get_top_matches(model, test_list):
    """
    Given an embedding model and list of tags, gets most similar results (most_similar method).
    To be run on one 
    """
#     ref_list = []
    matches = {}
#     not_found = 0
    for lstring in test_list:
        tagset = []
        try:
            match = model.wv.most_similar(lstring)
    #         print(ls, ' : ', match)
#             ref_list.append(lstring)
            for tag in range(len(match)):
                tagset.append(match[tag][0])

#             ref_list.append(tagset)
            matches[lstring] = tagset
        except KeyError:
    #         print(ls, ' : ','NOT_FOUND')
            pass
#     print("Not found", not_found)
    return matches

# search_list = rare
# get_top_matches(model, search_list)

In [19]:
def test_tag(test_dictionary):
    """
    Selects one random key from dictionary and determines if any values for that key
    match any other keys in the dictionary (in other words, whether the model's
    suggestion for a given tag matches any existing tags for the same asset).
    """
    import random
    rand = random.randint(0, len(test_dictionary) - 1)
    keylist = list(test_dictionary.keys())
    key = keylist[rand]
    suggestions = test_dictionary[key]
#     print(key, sugestions)
    matches = 0
    for suggestion in suggestions:
        for key in keylist:
            if suggestion == key:
#                 print("MATCH!", suggestion,  key)
                return("MATCH")
    return("NO MATCH")
    

In [80]:
#get embedding model and compute loss (for use in hyperparameter tuning)
import w2v_model
import time

for i in range(1,2):
    print(i)
    start = time.time()
    epochs = i
    vec_size = 10
    window = 5
    model = w2v_model.retrieve_model_na(epochs, vec_size, window)
    end = time.time()
    print(end - start)
    loss = model.get_latest_training_loss()
    # perplexity = 2**loss
    print(loss)

1
30.961588144302368
4410028.5


In [81]:
# get lemmatized tag df with 1 row per asset and each tag in a separate column, covert to list of lists
test = w2v_model.retrieve_expanded_query()
test_vals = test.values[1:100]

#use top_matches method to create a dictionary of related tags suggested by embedding model
asset_dicts = []
start = time.time()
for i in range(len(test_vals)):
#     print("remaining:", len(test_vals) - i)
    test_list = test_vals[i][test_vals[i] != None]
#     rate.append(test_list)
# #     print(test_list)
    top_matches = get_top_matches(model, test_list)
    asset_dicts.append(top_matches)
# #     str(test_list)
end = time.time()
print(end - start)
# asset_dicts

1.686035394668579


In [77]:
test_vals

array([['<dig><dig><dig><dig><dig><dig><dig><dig><dig>', 'boxing',
        'sport equipment', ..., None, None, None],
       ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>', 'red', 'night',
        ..., None, None, None],
       ['backstage',
        '<dig><dig><dig><dig><dig><dig><dig><dig><dig> <dig><dig><dig><dig> k love fan award backstage',
        'original', ..., None, None, None],
       ...,
       ['bfselects ftp', '<dig><dig><dig><dig><dig><dig><dig><dig><dig>',
        'plastic bottle', ..., None, None, None],
       ['<dig><dig><dig><dig><dig><dig><dig><dig><dig>', 'red',
        'fashion', ..., None, None, None],
       ['film industry', '<dig><dig><dig><dig><dig><dig><dig><dig><dig>',
        'led display', ..., None, None, None]], dtype=object)

In [79]:
#use test_tag method to determine useful suggestions
asset_results = []
for i in asset_dicts:
    asset_results.append(test_tag(i))
asset_results.count("MATCH")/len(asset_results)
# asset_results.count("MATCH")

0.7302473024730247

Next steps:  
    use lm1.ipynb